In [1]:
from kfp.v2 import dsl, compiler

import google.cloud.aiplatform as vertex
from google_cloud_pipeline_components import aiplatform as vc

In [2]:
REGION = 'us-central1'
PROJECT_ID = 'mitochondrion-project-344303'

BUCKET_NAME = 'gs://mitochondrion-bucket-sg'
PIPELINE_ROOT = f'{BUCKET_NAME}/pipelines'
PIPELINE_NAME = 'facemask-automl-pipeline'

JSON_FILE = 'facemask_automl_pipeline.json'

In [4]:
@dsl.pipeline(name=PIPELINE_NAME, pipeline_root=PIPELINE_ROOT)
def pipeline(
    display_name: str = PIPELINE_NAME,
    project: str = PROJECT_ID,
    annotation_file: str = 'gs://mitochondrion-bucket-us/datasets/facemask/annotations/train.csv',
):

    dataset = vc.ImageDatasetCreateOp(
        project=project,
        display_name=display_name,
        gcs_source=annotation_file,
        import_schema_uri=vertex.schema.dataset.ioformat.image.bounding_box
    )

    model_training = vc.AutoMLImageTrainingJobRunOp(
        project=project,
        display_name=display_name,
        dataset=dataset.outputs['dataset'],
        prediction_type='object_detection'
    )

    endpoint = vc.EndpointCreateOp(
        project=project,
        location=REGION,
        display_name=display_name,
    )

    vc.ModelDeployOp(
        model=model_training.outputs['model'],
        endpoint=endpoint.outputs['endpoint'],
        automatic_resources_min_replica_count=1,
        automatic_resources_max_replica_count=1
    )

In [5]:
compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path=JSON_FILE
)

/home/mitbal/miniconda3/lib/python3.9/site-packages/kfp/v2/compiler/compiler.py:1263: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  warnings.warn(


In [6]:
from google.cloud import aiplatform

job = aiplatform.PipelineJob(
    display_name = PIPELINE_NAME,
    template_path = JSON_FILE,
    enable_caching = True,
    project = PROJECT_ID,
    location = REGION,
    parameter_values = {
        'annotation_file': 'gs://mitochondrion-bucket-us/project/facemask/2022-01-01/facemask_annotations.csv'
    }
)

# job.submit()
job.submit(service_account='default@mitochondrion-project-344303.iam.gserviceaccount.com')


INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/832137092875/locations/us-central1/pipelineJobs/facemask-automl-pipeline-20220329151247
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/832137092875/locations/us-central1/pipelineJobs/facemask-automl-pipeline-20220329151247')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/facemask-automl-pipeline-20220329151247?project=832137092875
